In [2]:
import numpy as np
from astropy.table import Table, join
import fitsio
import os, sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits

import sys
sys.path.insert(0, '/global/homes/s/severett/repos/balutils/')
from balutils import stacked_catalogs as sc

import seaborn as sb
plt.style.use('seaborn')
sb.set_context("notebook", font_scale=1.5)

In [10]:
ls /project/projectdirs/des/severett/Balrog/run2/inputs

BALROG_RUN2_DEEP_CAT_FLAG0INVHS1BDFLT254v4.fits


In [12]:
run = 'y3-merged'
ver = 1.2

h = fitsio.read_header('/project/projectdirs/des/severett/Balrog/run2/inputs/BALROG_RUN2_DEEP_CAT_FLAG0INVHS1BDFLT254v4.fits', ext=1)
print(h)


XTENSION= 'BINTABLE'           / binary table extension
BITPIX  =                    8 / array data type
NAXIS   =                    2 / number of array dimensions
NAXIS1  =                  576 / length of dimension 1
NAXIS2  =              1230145 / length of dimension 2
PCOUNT  =                    0 / number of group parameters
GCOUNT  =                    1 / number of groups
TFIELDS =                   25 / number of table fields
TTYPE1  = 'id'                 / 
TFORM1  = 'J'                  / 
TTYPE2  = 'flags'              / 
TFORM2  = 'J'                  / 
TTYPE3  = 'bdf_flags'          / 
TFORM3  = 'J'                  / 
TTYPE4  = 'ra'                 / 
TFORM4  = 'D'                  / 
TTYPE5  = 'dec'                / 
TFORM5  = 'D'                  / 
TTYPE6  = 'psf_mag'            / 
TFORM6  = '4D'                 / 
TTYPE7  = 'bdf_T'              / 
TFORM7  = 'D'                  / 
TTYPE8  = 'bdf_T_err'          / 
TFORM8  = 'D'                  / 
TTYPE9  = 'bdf

In [ ]:
run = 'y3-merged'
ver = 1.2

match_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_matched_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)
det_file = '/project/projectdirs/des/severett/Balrog/{}/stacked_catalogs/{}/sof/balrog_detection_catalog_sof_{}_v{}.fits'.format(run, ver, run, ver)

match_cols = ['bal_id', 'true_id', 'meas_cm_mag_deredden',  'true_bdf_mag_deredden', 'true_gap_flux_fwhm4asec', 'meas_gapflux',
              'meas_cm_fracdev', 'true_bdf_fracdev', 'meas_cm_T', 'true_bdf_T', 'meas_cm_mag', 'true_bdf_mag', 'meas_cm_flux_s2n', 'meas_cm_s2n_r',
              'true_bdf_g', 'true_bdf_flux_deredden', 'true_bdf_flux_err', 'meas_cm_flux_cov']
det_cols = ['bal_id', 'meas_FLAGS_GOLD_SOF_ONLY', 'flags_foreground', 'flags_footprint', 'flags_badregions', 'match_flag_1.5_asec',
            'meas_EXTENDED_CLASS_SOF', 'ext_mag']
# sof = sc.BalrogMatchedCatalog(match_file, det_file, match_cols=match_cols, match_type='sof_only', vb=True)

print('Loading matched file...')
match = Table(fitsio.read(match_file, columns=match_cols))
print('Loading det file...')
det = Table(fitsio.read(det_file, columns=det_cols))
print('joining...')
sof = join(match, det, keys='bal_id', join_type='inner')
assert len(match) == len(sof)

# Match to Ian's DF classifier
df_file = 'cats/ugriz-mof02-JHK-extcorr_27May20_kNN_class.fits'
df = Table.read(df_file)
df.rename_column('id', 'true_id')

print('Joining with df classifier...')
sof = join(sof, df, keys='true_id', join_type='left')

cuts = np.where(
    (sof['meas_FLAGS_GOLD_SOF_ONLY'] < 2) &
    (sof['flags_foreground'] == 0 ) &
    (sof['flags_footprint'] == 1) &
    (sof['flags_badregions'] < 2) &
    (sof['meas_EXTENDED_CLASS_SOF'] > 1) &
    (sof['match_flag_1.5_asec'] < 2)
)

print('Making cuts...')
sof = sof[cuts]